In [2]:


import numpy as np
import pandas as pd



In [3]:


df_train = pd.read_csv('train.csv', encoding='utf-8',nrows=200000)
df_train['id'] = df_train['id'].apply(str)



In [4]:


df_all = df_train
df_all['question1'].fillna('', inplace=True)
df_all['question2'].fillna('', inplace=True)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import itertools


In [6]:
counts_vectorizer = CountVectorizer(max_features=10000-1).fit(
    itertools.chain(df_all['question1'], df_all['question2']))
other_index = len(counts_vectorizer.vocabulary_)


In [7]:
print(other_index)
print(counts_vectorizer.vocabulary_)

9999
{'jee': 4893, 'macbeth': 5428, 'center': 1636, 'hulk': 4364, 'plato': 6766, 'grammar': 3975, 'version': 9554, 'romeo': 7717, 'adaptations': 352, 'scrapping': 7931, 'using': 9464, 'gadgets': 3786, 'unlikely': 9398, 'mother': 5903, 'psychopaths': 7145, 'hospitals': 4325, 'ceo': 1643, 'published': 7154, 'war': 9694, 'shake': 8087, 'planet': 6751, 'meanings': 5626, 'famous': 3417, 'salman': 7823, 'fish': 3570, 'bengali': 1121, 'point': 6799, 'audi': 868, 'entrepreneur': 3168, 'slowly': 8289, 'arduino': 716, 'assessment': 797, 'dye': 2943, 'imagine': 4483, 'shot': 8151, 'catering': 1601, 'lunch': 5414, 'taiwan': 8828, 'quickbooks': 7228, 'xi': 9938, 'roommate': 7721, 'inmate': 4627, 'medical': 5646, 'practical': 6906, 'habit': 4059, 'picking': 6705, 'elderly': 3046, 'mid': 5746, 'percentile': 6609, 'toppers': 9130, 'days': 2450, 'openings': 6324, 'manchester': 5501, 'revolve': 7641, 'engineering': 3141, 'edu': 3009, 'driven': 2887, 'effectively': 3019, 'sap': 7843, 'together': 9099, 'o

In [8]:
print(df_all['question1'].shape)

(200000,)


In [9]:
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [10]:
print(counts_vectorizer.token_pattern)
print(counts_vectorizer)

(?u)\b\w\w+\b
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=9999, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [11]:


words_tokenizer = re.compile(counts_vectorizer.token_pattern)
#print(words_tokenizer)
t=df_all[0:1]['question1']
print(t)



0    What is the step by step guide to invest in sh...
Name: question1, dtype: object


In [27]:
def create_padded_seqs(texts, max_len=10):
    seqs = texts.apply(lambda s: 
        [counts_vectorizer.vocabulary_[w] if w in counts_vectorizer.vocabulary_ else other_index
         for w in words_tokenizer.findall(s.lower())])
    print(seqs)
    print(len(seqs))
    print(seqs.shape)
    return pad_sequences(seqs, maxlen=max_len)

In [28]:
sample=create_padded_seqs(t)
print(sample)

0    [9786, 4815, 8983, 8558, 1470, 8558, 4033, 909...
Name: question1, dtype: object
1
(1,)
[[1470 8558 4033 9093 4764 4531 8097 5541 4531 4563]]


In [14]:
X1_train, X1_val, X2_train, X2_val, y_train, y_val = \
    train_test_split(create_padded_seqs(df_all[df_all['id'].notnull()]['question1']), 
                     create_padded_seqs(df_all[df_all['id'].notnull()]['question2']),
                     df_all[df_all['id'].notnull()]['is_duplicate'].values,
                     stratify=df_all[df_all['id'].notnull()]['is_duplicate'].values,
                     test_size=0.3, random_state=1989)

0         [9786, 4815, 8983, 8558, 1470, 8558, 4033, 909...
1         [9786, 4815, 8983, 8584, 6263, 9999, 9999, 999...
2         [4345, 1520, 4552, 8983, 8433, 6263, 5984, 471...
3         [9814, 547, 5683, 9559, 5361, 4345, 1520, 8356...
4         [9800, 6306, 2768, 4531, 9721, 9999, 8684, 782...
5         [817, 547, 1554, 8702, 1541, 5885, 591, 1541, ...
6                                        [8155, 1465, 9999]
7                            [4345, 1520, 1047, 3937, 9999]
8                      [9794, 2801, 9968, 9456, 4665, 6263]
9          [5917, 1981, 1520, 4061, 5984, 1695, 9999, 9999]
10        [5720, 9093, 3543, 8042, 6263, 9999, 9464, 999...
11          [4345, 2801, 7339, 591, 3543, 5984, 9976, 1960]
12               [9786, 1520, 5475, 6697, 2973, 9093, 5185]
13               [9786, 9708, 9972, 3568, 8077, 3323, 5274]
14        [9786, 717, 8983, 5158, 9093, 1673, 9972, 8543...
15        [9786, 9911, 9270, 6958, 5623, 3651, 2371, 471...
16                                 [9786

In [23]:
import keras.layers as lyr
from keras.models import Model


In [29]:
print(X1_train.shape)
print(X2_train.shape)
print(X1_val.shape)
print(X2_val.shape)
print(len(y_train))
print(X1_train.shape[1:])
print(X1_train.max())


(140000, 10)
(140000, 10)
(60000, 10)
(60000, 10)
140000
(10,)
9999


In [25]:
input1_tensor = lyr.Input(X1_train.shape[1:])
input2_tensor = lyr.Input(X2_train.shape[1:])

words_embedding_layer = lyr.Embedding(X1_train.max() + 1, 100)
seq_embedding_layer = lyr.LSTM(256, activation='tanh')

seq_embedding = lambda tensor: seq_embedding_layer(words_embedding_layer(tensor))


In [30]:
merge_layer = lyr.multiply([seq_embedding(input1_tensor), seq_embedding(input2_tensor)])

In [31]:

dense1_layer = lyr.Dense(16, activation='sigmoid')(merge_layer)
ouput_layer = lyr.Dense(1, activation='sigmoid')(dense1_layer)
model = Model([input1_tensor, input2_tensor], ouput_layer)

model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 100)      1000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 256)          365568      embedding_1[0][0]                
          

In [32]:
model.fit([X1_train, X2_train], y_train, 
          validation_data=([X1_val, X2_val], y_val), 
          batch_size=128, epochs=6, verbose=2)

Train on 140000 samples, validate on 60000 samples
Epoch 1/6
 - 298s - loss: 0.5364 - val_loss: 0.5024
Epoch 2/6
 - 285s - loss: 0.4581 - val_loss: 0.4742
Epoch 3/6
 - 280s - loss: 0.4011 - val_loss: 0.4731
Epoch 4/6
 - 275s - loss: 0.3517 - val_loss: 0.4798
Epoch 5/6
 - 275s - loss: 0.3069 - val_loss: 0.5012
Epoch 6/6
 - 276s - loss: 0.2657 - val_loss: 0.5205


In [33]:
features_model = Model([input1_tensor, input2_tensor], merge_layer)
features_model.compile(loss='mse', optimizer='adam')

In [34]:
F_train = features_model.predict([X1_train, X2_train], batch_size=128)
F_val = features_model.predict([X1_val, X2_val], batch_size=128)


In [35]:
print(X1_val.shape)

(60000, 10)


In [36]:
print(X2_val.shape)

(60000, 10)


In [37]:
print(y_val)

[1 1 0 ... 1 0 1]


In [38]:
print(y_val.shape)

(60000,)


In [26]:
print(F_val.shape
     )

(60000, 256)


In [39]:
import xgboost as xgb

In [40]:
dTrain = xgb.DMatrix(F_train, label=y_train)
dVal = xgb.DMatrix(F_val, label=y_val)

In [43]:
print(dTrain)



In [44]:
xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.1, 
    'max_depth': 9,
    'subsample': 0.9,
    'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'min_child_weight': 5,
    'silent': 1
}
bst = xgb.train(xgb_params, dTrain, 1000,  [(dTrain,'train'), (dVal,'val')], 
                verbose_eval=10, early_stopping_rounds=10)

[0]	train-logloss:0.643751	val-logloss:0.657065
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 10 rounds.
[10]	train-logloss:0.397973	val-logloss:0.498792
[20]	train-logloss:0.305486	val-logloss:0.466789
[30]	train-logloss:0.256959	val-logloss:0.464825
Stopping. Best iteration:
[25]	train-logloss:0.277739	val-logloss:0.463181



In [45]:
X1_test = create_padded_seqs(df_all[df_all['id'].notnull()]['question1'])
X2_test = create_padded_seqs(df_all[df_all['id'].notnull()]['question2'])


0         [9786, 4815, 8983, 8558, 1470, 8558, 4033, 909...
1         [9786, 4815, 8983, 8584, 6263, 9999, 9999, 999...
2         [4345, 1520, 4552, 8983, 8433, 6263, 5984, 471...
3         [9814, 547, 5683, 9559, 5361, 4345, 1520, 8356...
4         [9800, 6306, 2768, 4531, 9721, 9999, 8684, 782...
5         [817, 547, 1554, 8702, 1541, 5885, 591, 1541, ...
6                                        [8155, 1465, 9999]
7                            [4345, 1520, 1047, 3937, 9999]
8                      [9794, 2801, 9968, 9456, 4665, 6263]
9          [5917, 1981, 1520, 4061, 5984, 1695, 9999, 9999]
10        [5720, 9093, 3543, 8042, 6263, 9999, 9464, 999...
11          [4345, 2801, 7339, 591, 3543, 5984, 9976, 1960]
12               [9786, 1520, 5475, 6697, 2973, 9093, 5185]
13               [9786, 9708, 9972, 3568, 8077, 3323, 5274]
14        [9786, 717, 8983, 5158, 9093, 1673, 9972, 8543...
15        [9786, 9911, 9270, 6958, 5623, 3651, 2371, 471...
16                                 [9786

In [47]:
F_test = features_model.predict([X1_test, X2_test], batch_size=128)

In [48]:
print(F_test.shape
     )

(200000, 256)


In [49]:


dTest = xgb.DMatrix(F_test)



In [50]:


df_sub = pd.DataFrame({
        'test_id': df_all[df_all['id'].notnull()]['id'].values,
        'is_duplicate': bst.predict(dTest, ntree_limit=bst.best_ntree_limit)
    }).set_index('test_id')



In [51]:
print(df_sub.shape)

(200000, 1)


In [52]:
print(df_sub)

         is_duplicate
test_id              
0            0.197262
1            0.139196
2            0.204831
3            0.053014
4            0.069156
5            0.326160
6            0.039670
7            0.671783
8            0.335513
9            0.092407
10           0.121424
11           0.390772
12           0.669317
13           0.821449
14           0.115981
15           0.928522
16           0.609026
17           0.084323
18           0.941369
19           0.372601
20           0.655989
21           0.444477
22           0.621413
23           0.047902
24           0.746103
25           0.059455
26           0.128411
27           0.084424
28           0.901338
29           0.900498
...               ...
199970       0.629979
199971       0.940245
199972       0.092758
199973       0.101131
199974       0.049798
199975       0.349497
199976       0.515820
199977       0.117980
199978       0.862657
199979       0.483382
199980       0.416989
199981       0.724529
199982    

In [53]:
df_sub.head()


,is_duplicate
test_id,
0,0.197262
1,0.139196
2,0.204831
3,0.053014
4,0.069156


In [54]:
y_svr_predicted=np.around(df_sub)
len(y_svr_predicted)
print(y_svr_predicted)

         is_duplicate
test_id              
0                 0.0
1                 0.0
2                 0.0
3                 0.0
4                 0.0
5                 0.0
6                 0.0
7                 1.0
8                 0.0
9                 0.0
10                0.0
11                0.0
12                1.0
13                1.0
14                0.0
15                1.0
16                1.0
17                0.0
18                1.0
19                0.0
20                1.0
21                0.0
22                1.0
23                0.0
24                1.0
25                0.0
26                0.0
27                0.0
28                1.0
29                1.0
...               ...
199970            1.0
199971            1.0
199972            0.0
199973            0.0
199974            0.0
199975            0.0
199976            1.0
199977            0.0
199978            1.0
199979            0.0
199980            0.0
199981            1.0
199982    

In [55]:

y_test=df_all['is_duplicate']
print(y_test)

0         0
1         0
2         0
3         0
4         0
5         1
6         0
7         1
8         0
9         0
10        0
11        1
12        1
13        1
14        0
15        1
16        1
17        0
18        1
19        0
20        1
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        1
         ..
199970    1
199971    1
199972    0
199973    1
199974    0
199975    0
199976    1
199977    0
199978    1
199979    0
199980    0
199981    1
199982    0
199983    0
199984    0
199985    0
199986    1
199987    0
199988    1
199989    0
199990    0
199991    1
199992    1
199993    1
199994    1
199995    1
199996    0
199997    0
199998    1
199999    0
Name: is_duplicate, Length: 200000, dtype: int64


In [56]:
print(y_test)
y=np.array(y_test)


0         0
1         0
2         0
3         0
4         0
5         1
6         0
7         1
8         0
9         0
10        0
11        1
12        1
13        1
14        0
15        1
16        1
17        0
18        1
19        0
20        1
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        1
         ..
199970    1
199971    1
199972    0
199973    1
199974    0
199975    0
199976    1
199977    0
199978    1
199979    0
199980    0
199981    1
199982    0
199983    0
199984    0
199985    0
199986    1
199987    0
199988    1
199989    0
199990    0
199991    1
199992    1
199993    1
199994    1
199995    1
199996    0
199997    0
199998    1
199999    0
Name: is_duplicate, Length: 200000, dtype: int64


In [57]:
print(y_svr_predicted)

         is_duplicate
test_id              
0                 0.0
1                 0.0
2                 0.0
3                 0.0
4                 0.0
5                 0.0
6                 0.0
7                 1.0
8                 0.0
9                 0.0
10                0.0
11                0.0
12                1.0
13                1.0
14                0.0
15                1.0
16                1.0
17                0.0
18                1.0
19                0.0
20                1.0
21                0.0
22                1.0
23                0.0
24                1.0
25                0.0
26                0.0
27                0.0
28                1.0
29                1.0
...               ...
199970            1.0
199971            1.0
199972            0.0
199973            0.0
199974            0.0
199975            0.0
199976            1.0
199977            0.0
199978            1.0
199979            0.0
199980            0.0
199981            1.0
199982    

In [58]:
print(y)

[0 0 0 ... 0 1 0]


In [59]:
y_svr_predicted

,is_duplicate
test_id,
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,1.0
8,0.0


In [60]:
f=np.array(y_svr_predicted["is_duplicate"])
print(f)


[0. 0. 0. ... 0. 1. 1.]


In [61]:
t=(y == f).sum()
print((y == f).sum())
print(t*100/len(y))

174962
87.481
